In [20]:
import pandas as pd

from thresh import load_interface
import os, getpass, json, random

In [21]:
random.seed(42)

In [22]:
df = pd.read_csv("/home/li.mil/hierarchical-mds/human_eval/csv/preference_ranking.csv")
df

,label,output
0,"At present, there is insufficient RCT evidence...",Output A: The provided articles suggest that b...
1,There is inadequate evidence to draw strong co...,Output A: The evidence from the provided artic...
2,Collaborative care is associated with signific...,Output A: The articles provide evidence that c...
3,Administration of systemic prophylactic antibi...,Output A: The evidence suggests that systemic ...
4,The limited available evidence suggests folate...,Output A: The articles suggest that folate sup...
...,...,...
465,There was evidence from three pooled RCTs that...,Output A: The evidence from randomized trials ...
466,Antibiotic prophylaxis is effective in prevent...,Output A: The evidence from the provided artic...
467,The practice of surgical and chemical sympathe...,Output A: The evidence from the provided artic...
468,There is not sufficient evidence to support or...,Output A: The evidence from the provided artic...


In [23]:
df.rename(columns={'label':'source', 'output': 'target'}, inplace=True)

In [24]:
df.head()

,source,target
0,"At present, there is insufficient RCT evidence...",Output A: The provided articles suggest that b...
1,There is inadequate evidence to draw strong co...,Output A: The evidence from the provided artic...
2,Collaborative care is associated with signific...,Output A: The articles provide evidence that c...
3,Administration of systemic prophylactic antibi...,Output A: The evidence suggests that systemic ...
4,The limited available evidence suggests folate...,Output A: The articles suggest that folate sup...


In [25]:
# Add an attention check
sampled = df.sample(10, random_state=42, replace=False)

In [26]:
line = pd.DataFrame({"source": "Researchers sometimes adopt attention check questions that ask participants to think back to an earlier part of the survey and recall specific information. Read the 'Generated Summaries' section, and follow the instructions exactly.", "target": "Select the order for Outputs A, B, C as the following: 3, 2, 1"}, index=[-1])
sampled = pd.concat(
    [sampled.iloc[:5],
     line,
     sampled.iloc[5:]
    ]
)

# line = pd.DataFrame({"source": "Researchers sometimes adopt attention check questions that ask participants to think back to an earlier part of the survey and recall specific information. Read the 'Generated Summaries' section, and follow the instructions exactly.", "target": "Select the order for Outputs B, A, C as the following: 3, 2, 1"}, index=[-1])
# sampled = pd.concat(
#     [sampled.iloc[:41],
#      line,
#      sampled.iloc[41:]
#     ]
# )

In [27]:
sampled.to_json("cochrane.json", orient='records')

with open("indices.txt", "w") as f:
    json.dump(list(sampled.index), f)

In [28]:
interface = load_interface('interface.yml')

In [29]:
local_template_path = "interface.yml"
local_data_path = "cochrane.json"
repository_name = "prolific-deployment-ml"

In [30]:
if os.path.exists('.git_credentials'):
    # This will read the API token from a local 
    github_username = "datalabNU"
    github_token = open('.git_credentials', 'r').read()
else:
    github_username = input("Please enter your GitHub username: ")
    github_token = getpass.getpass("Please enter your GitHub access token (register a token at github.com/settings/tokens): ")

### Template 1

In [17]:
template_path = f"{github_username}/{repository_name}/master/{local_template_path}"
data_path = f"{github_username}/{repository_name}/master/{local_data_path}"

print(f"Hosted template: {template_path}")
print(f"Hosted data: {data_path}")

Hosted template: datalabNU/prolific-deployment-ml/master/interface.yml
Hosted data: datalabNU/prolific-deployment-ml/master/cochrane.json


In [18]:
study_url = f"https://thresh.tools/?gh={template_path}"
study_url = study_url + "&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}"

In [19]:
print(f"View your hosted template here:\t\t  {study_url}")

study_url = f"{study_url}&d={data_path}"

print(f"Your study (with data) will be linked to: {study_url}")

View your hosted template here:		  https://thresh.tools/?gh=datalabNU/prolific-deployment-ml/master/interface.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}
Your study (with data) will be linked to: https://thresh.tools/?gh=datalabNU/prolific-deployment-ml/master/interface.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}&d=datalabNU/prolific-deployment-ml/master/cochrane.json


### Template 2

In [24]:
template_path = f"{github_username}/{repository_name}/master/interface2.yml"
data_path = f"{github_username}/{repository_name}/master/{local_data_path}"

print(f"Hosted template: {template_path}")
print(f"Hosted data: {data_path}")

Hosted template: datalabNU/prolific-deployment-ml/master/interface2.yml
Hosted data: datalabNU/prolific-deployment-ml/master/cochrane.json


In [25]:
study_url = f"https://thresh.tools/?gh={template_path}"
study_url = study_url + "&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}"

In [52]:
print(f"View your hosted template here:\t\t  {study_url}")

study_url = f"{study_url}&d={data_path}"

print(f"Your study (with data) will be linked to: {study_url}")

View your hosted template here:		  https://thresh.tools/?gh=datalabNU/prolific-deployment-ml/master/interface2.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}&TESTING
Your study (with data) will be linked to: https://thresh.tools/?gh=datalabNU/prolific-deployment-ml/master/interface2.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}&TESTING&d=datalabNU/prolific-deployment-ml/master/cochrane.json


### Template 3

In [42]:
template_path = f"{github_username}/{repository_name}/master/interface3.yml"
data_path = f"{github_username}/{repository_name}/master/{local_data_path}"

print(f"Hosted template: {template_path}")
print(f"Hosted data: {data_path}")

Hosted template: datalabNU/prolific-deployment-ml/master/interface3.yml
Hosted data: datalabNU/prolific-deployment-ml/master/cochrane.json


In [45]:
study_url = f"https://thresh.tools/?gh={template_path}"
study_url = study_url + "&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}"

In [46]:
print(f"View your hosted template here:\t\t  {study_url}")

study_url = f"{study_url}&d={data_path}"

print(f"Your study (with data) will be linked to: {study_url}")

View your hosted template here:		  https://thresh.tools/?gh=datalabNU/prolific-deployment-ml/master/interface3.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}
Your study (with data) will be linked to: https://thresh.tools/?gh=datalabNU/prolific-deployment-ml/master/interface3.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}&d=datalabNU/prolific-deployment-ml/master/cochrane.json
